<h2> Taller 7 - Cinemática Inversa con Sympy</h2>

<br>Requiere que los archivos 'sim.py', 'simConst.py', 'remoteapi.dll' estén alojados en la misma carpeta que este cuaderno de Jupyter.
<br> Desde CoppeliaSim, abrir la escena MTB_IK.ttt


In [ ]:
# importamos las librerías necesarias
import sim          # librería para conectar con CoppeliaSim
import sympy as sp  # librería para cálculo simbólico

### 0. Verificamos que todo esté funcionando... 

El robot cuenta con una vensosa en el extremo. Ésta se activa automáticamente al acercarse al cubo (1 mm o menos) y se desactiva cuando el cubo se coloca sobre la plataforma (5 mm o menos).

In [ ]:
def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


In [ ]:
# Requerimos los manejadores para las articulaciones y el Dummy
clientID = connect(19999)

retCode,tip=sim.simxGetObjectHandle(clientID,'suctionPadSensor',sim.simx_opmode_blocking)
retCode,joint1=sim.simxGetObjectHandle(clientID,'MTB_joint1',sim.simx_opmode_blocking)
retCode,joint2=sim.simxGetObjectHandle(clientID,'MTB_joint2',sim.simx_opmode_blocking)
retCode,joint3=sim.simxGetObjectHandle(clientID,'MTB_joint3',sim.simx_opmode_blocking)
retCode,joint4=sim.simxGetObjectHandle(clientID,'MTB_joint4',sim.simx_opmode_blocking)
print(tip, joint1, joint2, joint3, joint4)

In [ ]:
# Enviamos las posiciones a las articulaciones
q = [0, 0, 0]

retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)


###  1. Preparamos el modelo del robot
Utilizaremos la librería sympy para obtener las acuaciones de cinemática directa y cinemática inversa.
<br> Para más detalles refiérase a la documentación de la librería:
<br> https://docs.sympy.org/latest/index.html

In [ ]:
# Comenzaremos el trabajo desde la descripción de Denavit-Hartenberg para el robot
#      theta     |      d      |      a      |    alpha
# ---------------------------------------------------------
#      q1        |    0.302    |    0.467    |     0
#      q2        |    -0.01    |    0.4005   |    180
#      0         |      q3     |     0       |     0
#      q4        |      0      |     0       |    180
# ---------------------------------------------------------
#      0         |    -0.058   |     0       |     0

In [ ]:
# Definimos una función para construir las matrices de transformación
# en forma simbóĺica a partir de los parámetros D-H

def symTfromDH(theta, d, a, alpha):
    # theta y alpha en radianes
    # d y a en metros
    Rz = sp.Matrix([[sp.cos(theta), -sp.sin(theta), 0, 0],
                   [sp.sin(theta), sp.cos(theta), 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    tz = sp.Matrix([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, d],
                   [0, 0, 0, 1]])
    ta = sp.Matrix([[1, 0, 0, a],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    Rx = sp.Matrix([[1, 0, 0, 0],
                   [0, sp.cos(alpha), -sp.sin(alpha), 0],
                   [0, sp.sin(alpha), sp.cos(alpha), 0],
                   [0, 0, 0, 1]])
    T = Rz*tz*ta*Rx
    return T

In [ ]:
T01 = symTfromDH(0, 0, 0, 0)
T01

### 2. Calculamos las expresiones de cinemática inversa ###

La matriz T contiene toda la información necesaria para calcular la cinemática inversa. En esta ocasión nos preocuparemos únicamente de las coordenadas de posición.

In [ ]:
# Definimos un punto de destino
x = 0.5
y = 0.5
z = 0.1

In [ ]:
# preparamos las ecuaciones transformando las expresiones
# de la forma <expresion = valor> a la forma <expresion - valor> = 0


In [ ]:
# enviamos los ángulos a las articulaciones
clientID = connect(19999)
retCode = sim.simxSetJointTargetPosition(clientID, joint1, q[0], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint2, q[1], sim.simx_opmode_oneshot)
retCode = sim.simxSetJointTargetPosition(clientID, joint3, q[2], sim.simx_opmode_oneshot)


In [ ]:
# y verificamos la posición del actuador
retCode,pos=sim.simxGetObjectPosition(clientID, tip, -1, sim.simx_opmode_blocking)
print(pos)

<h3> Taller 6 </h3>
Utilizando el procedimiento para el cálculo de la cinemática inversa, construya una secuencia de movimientos para colocar el cubo sobre la plataforma. Siga el siguiente procedimiento:<br><br>
1. Baje el actuador para recoger el cubo.<br>
2. Levante el cubo desde el piso.<br>
3. Lleve el cubo sobre la plataforma (manteniendo la altura, ubíquelo sobre las mismas coordenas x,y de la plataforma.<br>
4. Baje el cubo hasta ubicarlo a una distancia suficiente de la plataforma para soltar el cubo.<br>
5. Vuelva a la posición inicial.<br>
<br>
Sugerencia: obtenga automáticamente la posición del cubo (Rectangle) y la plataforma (target) y realice los cálculos a partir de estas coordenadas. Recuerde que las coordenadas que obtendrá serán las coordenadas del centro del objeto.

In [ ]:
#  escriba su código aquí  #